In [16]:
def jobs_manager():
    from IPython.lib.backgroundjobs import BackgroundJobManager
    from IPython.core.magic import register_line_magic
    from IPython import get_ipython

    jobs = BackgroundJobManager()

    @register_line_magic
    def job(line):
        ip = get_ipython()
        jobs.new(line, ip.user_global_ns)

    return jobs
import sys
import argparse
import random
from nile.api.v1 import (
    clusters,
    Record,
    filters as nf,
    aggregators as na,
    grouping as ng,
    extractors as ne,
    statface as ns
)
from qb2.api.v1 import (
    filters as sf,
    extractors as se,
    resources as sr
)
import nile
from urlparse import urlparse
import json
from itertools import product
from nile.api.v1 import datetime as nd
import re
import pandas as pd
import random
import string
from datetime import datetime
import numpy as np

In [2]:
cluster = clusters.YT(proxy="hahn.yt.yandex.net", pool="voice") \
                .env(templates=dict(
                                job_root=('home/voice/talamable/VA-274'),
                                dates='{2018-08-20..2018-08-26}'))

In [4]:
def parse_us(groups):
    import libra
    for key, records in groups:
        uuid = key.key
        if not uuid.startswith('uu/'):
            continue
        try:
            session = libra.ParseSession(records, 'blockstat.dict')
        except:
            continue
        for r in session:
            urls = {}
            if not r.IsA('TMobileAppYandexWebRequest'):
                continue
            if not ('query_source=dialog' in r.FullRequest):
                continue
            commercial = float(r.RelevValues.get('cm2', -1))
            results = r.GetMainBlocks()
            direct_results = 0
            direct_clicks = 0
            click_cost = 0
            wizard_results = {}
            for block in results:
                res = block.GetMainResult()
                if res.IsA('TDirectResult'):
                    if int(res.FraudBits) != 0:
                        continue
                    direct_results += 1
                    for click in block.GetClicks():
                        if not click.IsDynamic:
                            direct_clicks += 1
                            if click.GetProperties('TDirectClickProperties').EventCost is not None:
                                click_cost += click.GetProperties('TDirectClickProperties').EventCost
                elif res.IsA('TWizardResult') or res.IsA('TBlenderWizardResult'):
                    wizard_results[res.Name] = wizard_results.get(res.Name, 0) + \
                            sum([not x.IsDynamic for x in block.GetClicks()])
            yield Record(query=r.Query,
                         cm=commercial,
                         direct_results=direct_results,
                         direct_clicks=direct_clicks,
                         click_cost=click_cost,
                         wizard_results=json.dumps(wizard_results)
                        )

In [7]:
job = cluster.job()
us = job.table('//user_sessions/pub/search/daily/@dates/clean')
reqs = us.groupby('key').sort('subkey').reduce(parse_us,
                                                   files=[nile.files.RemoteFile('statbox/statbox-dict-last/blockstat.dict'),
                                                          nile.files.RemoteFile('statbox/resources/libra.so')
                                                         ],
                                                   memory_limit=4000
                                                  ) \
            .put('$job_root/wizard_direct_all_alice_week')
job.run()

VBox()

In [13]:
def all_wizards_names(records):
    for rec in records:
        wiz = json.loads(rec['wizard_results'])
        for w in wiz:
            yield Record(wiz=w, clicks=1 * (wiz[w] > 0))
job = cluster.job()
wn = job.table('$job_root/wizard_direct_all_alice_week') \
        .map(all_wizards_names) \
        .groupby('wiz') \
        .aggregate(num_req=na.count(),
                   clicks=na.sum('clicks')
                  ) \
        .sort('num_req') \
        .put('$job_root/wizard_names')
job.run()

VBox()

In [20]:
def prepare_results(groups):
    for key, records in groups:
        direct_clicks = 0.
        direct_cost = 0.
        direct_results = 0.
        direct_serps = 0.
        wizards = {}
        cm = []
        for rec in records:
            direct_clicks +=  rec['direct_clicks']
            direct_results += rec['direct_results']
            direct_cost += rec['click_cost']
            if rec['direct_results'] > 0:
                direct_serps += 1
            for w, cl in json.loads(rec['wizard_results']).iteritems():
                if w not in wizards:
                    wizards[w] = {'shows': 0., 'clicks': 0.}
                wizards[w]['shows'] += 1
                wizards[w]['clicks'] += (cl>0) * 1
            cm.append(rec['cm'])
        yield Record(query=key.query,
                     direct_clicks=direct_clicks,
                     direct_cost=direct_cost,
                     direct_results=direct_results,
                     direct_serps=direct_serps,
                     wizards=json.dumps(wizards),
                     cm=np.median(cm),
                     cnt=len(cm)
                    )
        

job = cluster.job()
wn = job.table('$job_root/wizard_direct_all_alice_week') \
        .groupby('query') \
        .reduce(prepare_results) \
        .put('$job_root/wizard_direct_aggregated_all_alice_week')
job.run()

VBox()